In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

from sklearn.svm import LinearSVC, SVC

from sklearn.metrics import f1_score, confusion_matrix, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# option d'affichage des résultats
#pd.set_option("display.max_row",130)
pd.set_option("display.max_columns",100)

In [ ]:
# les noms des variables

colonnes =(["IDENT", "TypeHabi", "GeoFaz_Scl", "GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Anciennete",
            "Logement_prix_m2", "Mediane_RFR", "Sexe", "Prenom_Phase_scl_H", "Prenom_Phase_scl_F",
            "Prenom_Phase_cl_H", "Prenom_Phase_cl_F", "Age_H", "Age_F", "Affinite_1", "Affinite_2",
            "Prenom_effectif", "Potentiel_prenom", "Onoma-Phase_Scl", "Onoma-Phase_CL", "Top_mon"])

In [ ]:
#data_orig = pd.read_csv("C:/Users/Christian/Desktop/my_data_FDFIFI19.txt",sep=" " , names = colonnes, low_memory=False, dtype=str )#, dtype='Int64')
data_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/my_data_FDFTOT21.txt", delim_whitespace="\t", header = None, names=colonnes, low_memory=False, dtype=str )#, dtype='Int64')
data = data_orig.copy()
data

,IDENT,TypeHabi,GeoFaz_Scl,GeoFaz_cl,ImmoFaz_Scl,ImmoFaz_cl,Anciennete,Logement_prix_m2,Mediane_RFR,Sexe,Prenom_Phase_scl_H,Prenom_Phase_scl_F,Prenom_Phase_cl_H,Prenom_Phase_cl_F,Age_H,Age_F,Affinite_1,Affinite_2,Prenom_effectif,Potentiel_prenom,Onoma-Phase_Scl,Onoma-Phase_CL,Top_mon
0,000000010010000001,02,81,08,91,09,06,21,21,01,71,41,07,04,12,04,20,02,02,01,51,05,1
1,000000020010000006,01,75,07,43,04,06,06,13,02,71,64,07,06,12,09,04,12,04,13,12,01,1
2,000000030010000011,01,76,07,43,04,06,06,16,03,54,81,05,08,07,12,06,06,04,18,36,03,2
3,000000040010000013,03,75,07,43,04,04,06,13,01,71,36,07,03,12,11,11,19,04,06,21,02,1
4,000000050010000015,02,33,03,44,04,06,07,12,01,71,42,07,04,12,05,14,03,03,01,14,01,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219994,010928800370109343,01,64,06,82,08,05,16,17,02,71,81,07,08,12,12,21,21,06,21,61,06,NaN
1219995,010928810370109344,03,61,06,64,06,06,14,01,04,44,81,04,08,05,12,01,05,01,03,54,05,NaN
1219996,010928820370109345,02,13,01,42,04,07,07,15,02,71,81,07,08,12,12,21,21,06,21,61,06,NaN
1219997,010928830370109346,02,31,03,82,08,08,16,19,02,71,45,07,04,12,03,02,02,03,01,44,04,NaN


In [ ]:
#nombre de modalités de chaque variables (dans un dataframe)
def nombre_mod(data2):
    #création d'un dataframe vide
    dff= pd.DataFrame(columns=["VARIABLE","NOMBRE_MOD", "NOMBRE_MOD_SANS_NA"])
    for i in range(data2.shape[1]): # sans compter les valeurs manquantes
        taille =len(data2.iloc[:,i].value_counts(dropna=False))
        taille_1 = len(data2.iloc[:,i].value_counts(dropna=True) )
        dff=dff.append({"VARIABLE": colonnes[i],"NOMBRE_MOD":taille, "NOMBRE_MOD_SANS_NA":taille_1 }, ignore_index=True)
    return dff

# sélection des corrélations les plus importantes
def variables_corr(tableau_corr, seuil):
    colonnes = tableau_corr.columns
    val_temp = 0 
    variables_corr_pos=[] ; variables_corr_pos_val=[]
    variables_corr_neg=[] ; variables_corr_neg_val=[]
    for i in range(tableau_corr.shape[0]):  
        for j in range(tableau_corr.shape[0]):
            if i<j: #i!=j:
                if (tableau_corr.iloc[i,j] > seuil):
                    variables_corr_pos.append(colonnes[j])
                    variables_corr_pos_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

                if (tableau_corr.iloc[i,j] <-seuil):
                    variables_corr_neg.append(colonnes[j])
                    variables_corr_neg_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

        if (len(variables_corr_pos) > 0) | (len(variables_corr_neg) >0) :
            print("variable : ", colonnes[i])
            if len(variables_corr_pos) > 0:    
                print("corr pos", variables_corr_pos)
                print("corr pos", variables_corr_pos_val)
                val_temp += 1

            if len(variables_corr_neg) >0:  
                print("corr neg", variables_corr_neg)
                print("corr neg", variables_corr_neg_val)
                val_temp += 1
            print("")
        variables_corr_pos = variables_corr_pos*0
        variables_corr_neg = variables_corr_pos*0

        variables_corr_pos_val = variables_corr_pos_val*0
        variables_corr_neg_val = variables_corr_pos_val*0
    if val_temp == 0:
        print("IL N'Y A PAS DE CORRELATION ENTRE LES VARIABLES PAR RAPPORT AU SEUIL PRIS")
        

def evaluation(model, X_train, y_train, X_test, y_test, scoring, plot):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("test set : ")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred)) 
    print("train set : ")
    print(confusion_matrix(y_train, model.predict(X_train)))  
    print(classification_report(y_train, model.predict(X_train) ))

    if plot == True:
        N, train_score, val_score = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = scoring, train_sizes = np.linspace(0.1, 1, 6) )

        plt.figure(figsize = (10,6))
        plt.plot(N, train_score.mean(axis=1), label ="train_score")
        plt.plot(N, val_score.mean(axis=1), label ="cross_validation_score")
        plt.legend()
    else :
        print("GRAPHIQUE NON DEMANDE")

# Comme il n'y a pas de valeurs manquantes (à la place il y a un nombre), le seul traitement qu'on fait c'est tranformer
# le type de variable (les variables continues deviennent qualitatives)
def data_processing(df):
    """ 
    for i in range(0,len(df.columns)):    
        df[df.columns[i]].fillna(100.0, inplace=True) #On remplace les données manquantes par 100
    for i in range(0,len(df.columns)):
        print(df[df.columns[i]].value_counts(dropna=False))    
    """
    
   #Transformation de toutes les variables en facteurs         
    for i in range(0,len(df.columns)):
        df[df.columns[i]] = df[df.columns[i]].astype('category')
    return df.to_numpy() #.as_matrix()


def vingtiles(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)
    
    # pour la colonne Top_mon :
    # remplissage de la dernière colonne par 2 (VALABLE QUE POUR CE DATA, MODIFIER POUR D'AUTRES)
    data_temp_t = data_orig.copy()
    # remplacement
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne total
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

def vingtiles_ind_sans_target(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    print(df_tous)
    print(df_tous["TRANCHES"].value_counts(ascending = True))

    # GERE L'ORDRE
    patra = np.unique(df_tous.iloc[:,2], return_counts=True)
    df_tranches_sans_target = pd.DataFrame([patra[0],patra[1]]).T
    df_tranches_sans_target.columns=["TRANCHES","COUNTS"]

    # affichage
    if affichage ==True:        
        print(df_tranches_sans_target)
    
    # enregistrement sous Excel
    if save == True:
        df_tranches_sans_target.to_excel(path)
    

# élimination des variables trop corrélées (choix manuel)
def elimination_val_corr(data):
    data = data.drop(columns=[""])
    return data

  


In [ ]:
####################################################################################
#                                 PREMIERS TESTS
####################################################################################


In [ ]:
##########################################
# SANS LES VARIABLES CORR  ET ENCODAGE DUMMIES
#########################################

In [ ]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F",
                "Onoma-Phase_Scl", "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe IFI+
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [ ]:
model_LinearSVC = LinearSVC(class_weight ="balanced", max_iter=10000, random_state =0 )
param = { "penalty": ["l1", "l2"], "C":[0.001, 0.01,0.1, 0.5,1, 1.5, 2, 10 ], "loss":["hinge", "squared_hinge"],
         "dual": [True, False]}

grid_LinearSVC = GridSearchCV(model_LinearSVC, param, cv = 4, scoring = "recall")
resultat_grid_LinearSVC = grid_LinearSVC.fit(train_X_s, train_Y_s)
resultat_grid_LinearSVC.best_params_


{'C': 0.001, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l1'}

In [ ]:
#final_model_LinearSVC = LinearSVC(C=0.001, penalty='l2', loss= 'squared_hinge', class_weight ="balanced",max_iter=10000 , random_state=0 ) 
final_model_LinearSVC = LinearSVC(C=0.001,dual=False, loss = "squared_hinge", penalty = "l1", class_weight = "balanced", max_iter=10000, random_state=0  )
evaluation(final_model_LinearSVC, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

TypeError: ignored

In [ ]:

quarantiles(model=final_model_LinearSVC, data_orig= data, affichage=True,
            save=False, path = "C:/Users/Christian/Desktop/sans_dons_encodage_dummies_logistic_regression.xlsx")

AttributeError: ignored

In [ ]:
#   POUR LE MODELE LINEARSVC, ON NE PEUT PAS CALCULER DES PROBABILITES

In [ ]:
###########################################################################
###########################################################################

In [ ]:
# test SVC
final_model_SVC = SVC(C=0.001, probability=True, cache_size=1000, class_weight = "balanced", max_iter=10000, random_state=0  )
evaluation(final_model_SVC, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 4679     0]
 [20744     0]]
              precision    recall  f1-score   support

           1       0.18      1.00      0.31      4679
           2       0.00      0.00      0.00     20744

    accuracy                           0.18     25423
   macro avg       0.09      0.50      0.16     25423
weighted avg       0.03      0.18      0.06     25423

train set : 
[[18777     0]
 [82915     0]]
              precision    recall  f1-score   support

           1       0.18      1.00      0.31     18777
           2       0.00      0.00      0.00     82915

    accuracy                           0.18    101692
   macro avg       0.09      0.50      0.16    101692
weighted avg       0.03      0.18      0.06    101692

GRAPHIQUE NON DEMANDE


In [ ]:
####################################################################################################
#
#                                       UNDER-SAMPLING  ET OVER-SAMPLING
#
####################################################################################################

In [ ]:
# Résultat du traitement sur le train set (under-sampling et over-sampling)
abc_test = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/sans_val_corr_spad_under_over_sampling.txt", delim_whitespace="\t", header = None, low_memory=False, dtype="Int64" ) #, dtype='Int64')
abc_np = abc_test.to_numpy(dtype="Int64")

X_resampled = abc_np[:,:-1]
y_resampled = abc_np[:, abc_np.shape[1] -1 ]

In [ ]:
X_resampled.shape, y_resampled.shape

((160026, 269), (160026,))

In [ ]:
final_model_SVC = SVC(C=1, class_weight = "balanced", max_iter=10000, random_state=0)
evaluation(final_model_SVC, X_resampled, y_resampled, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 1043  3636]
 [ 3729 17015]]
              precision    recall  f1-score   support

           1       0.22      0.22      0.22      4679
           2       0.82      0.82      0.82     20744

    accuracy                           0.71     25423
   macro avg       0.52      0.52      0.52     25423
weighted avg       0.71      0.71      0.71     25423

train set : 
[[64762 15251]
 [12708 67305]]
              precision    recall  f1-score   support

           1       0.84      0.81      0.82     80013
           2       0.82      0.84      0.83     80013

    accuracy                           0.83    160026
   macro avg       0.83      0.83      0.83    160026
weighted avg       0.83      0.83      0.83    160026

GRAPHIQUE NON DEMANDE


In [ ]:
final_model_SVC = SVC(C=10, class_weight = "balanced", max_iter=10000, random_state=0  )
evaluation(final_model_SVC, X_resampled, y_resampled, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 2206  2473]
 [ 7898 12846]]
              precision    recall  f1-score   support

           1       0.22      0.47      0.30      4679
           2       0.84      0.62      0.71     20744

    accuracy                           0.59     25423
   macro avg       0.53      0.55      0.51     25423
weighted avg       0.72      0.59      0.64     25423

train set : 
[[71470  8543]
 [27523 52490]]
              precision    recall  f1-score   support

           1       0.72      0.89      0.80     80013
           2       0.86      0.66      0.74     80013

    accuracy                           0.77    160026
   macro avg       0.79      0.77      0.77    160026
weighted avg       0.79      0.77      0.77    160026

GRAPHIQUE NON DEMANDE


In [ ]:
final_model_SVC = SVC(C=10, kernel="poly", class_weight = "balanced", max_iter=10000, random_state=0 )
evaluation(final_model_SVC, X_resampled, y_resampled, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 1898  2781]
 [ 7244 13500]]
              precision    recall  f1-score   support

           1       0.21      0.41      0.27      4679
           2       0.83      0.65      0.73     20744

    accuracy                           0.61     25423
   macro avg       0.52      0.53      0.50     25423
weighted avg       0.71      0.61      0.65     25423

train set : 
[[62117 17896]
 [25626 54387]]
              precision    recall  f1-score   support

           1       0.71      0.78      0.74     80013
           2       0.75      0.68      0.71     80013

    accuracy                           0.73    160026
   macro avg       0.73      0.73      0.73    160026
weighted avg       0.73      0.73      0.73    160026

GRAPHIQUE NON DEMANDE


In [ ]:
final_model_SVC = SVC(C=10, kernel="sigmoid", class_weight = "balanced", max_iter=10000, random_state=0 )
evaluation(final_model_SVC, X_resampled, y_resampled, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 1633  3046]
 [ 6271 14473]]
              precision    recall  f1-score   support

           1       0.21      0.35      0.26      4679
           2       0.83      0.70      0.76     20744

    accuracy                           0.63     25423
   macro avg       0.52      0.52      0.51     25423
weighted avg       0.71      0.63      0.67     25423

train set : 
[[47263 32750]
 [24098 55915]]
              precision    recall  f1-score   support

           1       0.66      0.59      0.62     80013
           2       0.63      0.70      0.66     80013

    accuracy                           0.64    160026
   macro avg       0.65      0.64      0.64    160026
weighted avg       0.65      0.64      0.64    160026

GRAPHIQUE NON DEMANDE


In [ ]:
final_model_SVC = SVC(C=10, kernel="sigmoid", probability=True, cache_size=2000 ,class_weight = "balanced", max_iter=1000, random_state=0 )
evaluation(final_model_SVC, X_resampled, y_resampled, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 3871   808]
 [17304  3440]]
              precision    recall  f1-score   support

           1       0.18      0.83      0.30      4679
           2       0.81      0.17      0.28     20744

    accuracy                           0.29     25423
   macro avg       0.50      0.50      0.29     25423
weighted avg       0.69      0.29      0.28     25423

train set : 
[[72782  7231]
 [67071 12942]]
              precision    recall  f1-score   support

           1       0.52      0.91      0.66     80013
           2       0.64      0.16      0.26     80013

    accuracy                           0.54    160026
   macro avg       0.58      0.54      0.46    160026
weighted avg       0.58      0.54      0.46    160026

GRAPHIQUE NON DEMANDE


In [ ]:

# TRAIN SET TEST SET
vingtiles(model = final_model_SVC, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Adaboost.xlsx")

                     IDENT     PROBA
108220  001082210010511190  0.179766
113201  001132020010523410  0.186521
113200  001132010010523410  0.186521
91247   000912480010404582  0.186521
3823    000038240010041200  0.189619
...                    ...       ...
125604  001256050010537952  0.840332
25299   000253000010330286  0.840332
125603  001256040010537952  0.840332
87161   000871620010401170  0.841397
110647  001106480010520247  0.841591

[127115 rows x 2 columns]
0.500000    12890
0.612985       53
0.666279       33
0.606169       33
0.639366       26
            ...  
0.584625        1
0.539291        1
0.418904        1
0.529169        1
0.490791        1
Name: PROBA, Length: 86625, dtype: int64


ValueError: ignored